In [299]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import function_library_assignment_1 as fnc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #filtering FutureWarnings - we don't need to worry about them in this case
#(This is run using Python 3.9)

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [300]:
## Reading data
gen_data = fnc.read_data('gen_data')
demand = fnc.read_data('system_demand')['System Demand']
load_distribution = fnc.read_data('load_distribution')
gen_costs = fnc.read_data('gen_costs')
line_data = fnc.read_data('line_data')
branch_matrix = fnc.read_data('branch_matrix')
wind_data = fnc.read_data('wind_data')
all_bid_prices = fnc.read_data('all_bid_prices')

GC = len(gen_data.index)
GW = len(wind_data.columns)
G = GC + GW
D = len(load_distribution.index)
N = 24 # number of buses

WF_NODES = [2, 4, 6, 15, 20, 22] # zero-indexed

t = 17 #hour

In [301]:
## Setting up demand for the hour: 
load = np.zeros(N)

#Saving the load for each bus in a numpy array accounting for the system load distribution
for n in load_distribution['Node'].unique():
    load[n-1] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * demand[t] #per-unitized load - remember that the data is not 0-indexed but the arrays are
load

array([100.719 ,  90.117 , 166.9815,  68.913 ,  66.2625, 127.224 ,
       116.622 , 159.03  , 161.6805, 180.234 ,   0.    ,   0.    ,
       246.4965, 180.234 , 294.2055,  92.7675,   0.    , 310.1085,
       169.632 , 119.2725,   0.    ,   0.    ,   0.    ,   0.    ])

In [302]:
# Set up bid prices
bid_prices = np.zeros(D)

for n in range(D):
    bid_prices[n] = all_bid_prices.loc[all_bid_prices['Load #'] == (n + 1), f't{t+1}']
    #print(f'Load {n+1} bid price: {bid_prices[n]} $/MWh')

In [303]:
# DA Hourly Model
direction = gb.GRB.MAXIMIZE #Min / Max

m = gb.Model() # Create a Gurobi model  

#============= Variables =============
p_G = m.addVars(G, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="P_G") #Note: wind farms can be curtailed AND P_min from the system should be disregarded to avoid having a mixed integer program
p_D = m.addVars(D, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="P_D") #Note: demands are elastic

#============= Objective function =============
# Set objective function (social welfare) - note that production cost of wind farms is assumed to be zero
obj = gb.quicksum(bid_prices[d] * p_D[d] for d in range(D)) - gb.quicksum(gen_costs['C ($/MWh)'][k] * p_G[k] for k in range(GC))
m.setObjective(obj, direction)

#============= Balance equation =============
m.addConstr(gb.quicksum(p_D[d] for d in range(D)) - gb.quicksum(p_G[g] for g in range(G)) == 0)

#============= Generator limits ============

#Upper limits
m.addConstrs(p_G[g] <= (gen_data['P max MW'].iloc[g]) for g in range(GC)) #conventinal generator upper limits

#Maximum wind power is the per-unitized output of the non-curtailed wind farm in the hour t
m.addConstrs(p_G[GC + g] <= wind_data.iloc[t, g] for g in range(GW)) #wind farm generator upper limits

#Lower limits (coded as constraints to be able to extract dual values for KKTs)
m.addConstrs(-p_G[g] <= 0 for g in range(G))

#============= Demand limits ===============
m.addConstrs(p_D[d] <= load[load != 0][d] for d in range(D)) #demand upper limits
m.addConstrs(-p_D[d] <= 0 for d in range(D))

#============= Display and run model =============
m.update()
#m.display()
m.optimize()

print('\nRuntime: %f ms' % (m.Runtime * 1e3))

da_market_price = m.Pi[0]

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 71 rows, 35 columns and 105 nonzeros
Model fingerprint: 0x1e501bff
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 70 rows and 15 columns
Presolve time: 0.00s
Presolved: 1 rows, 20 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8891123e+04   2.456544e+02   0.000000e+00      0s
       1    3.2346961e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.234696121e+04


In [304]:
# Result anaysis

# Print solutions
if m.status == gb.GRB.OPTIMAL:
    results = {}
    br = pd.DataFrame(data=np.zeros(G), columns=['p_G'])
    br['Node'] = 0 #initialize
    br['Dispatched Percentage'] = 0
    br['Type'] = 'Conventional'

    load_results = pd.DataFrame(data=np.zeros(D), columns=['p_D'])
    load_results['Node'] = load_distribution['Node'].values
    load_results['Maximum Demand'] = load[load != 0]
    load_results['Covered Percentage'] = 0

    constraints = m.getConstrs()
    # The constraint dual value of the current solution (also known as the shadow price)... https://www.gurobi.com/documentation/9.5/refman/pi.html
    dual_values = [constraints[k].Pi for k in range(len(constraints))] 

    for i in range(G):
        print(p_G[i].VarName + ": %.2f MW" % p_G[i].x)
        generator_outputs.loc[generator_outputs.index == i, 'p_G'] = p_G[i].x

        if i < GC:
            generator_outputs.loc[generator_outputs.index == i, 'Node'] = gen_data.loc[gen_data.index == i, 'Node'].values #save node
            gen_limit = gen_data.loc[gen_data.index == i, 'P max MW']
            generator_outputs.loc[generator_outputs.index == i, 'Dispatched Percentage'] = (p_G[i].x / gen_limit) * 100
        else:
            generator_outputs.loc[generator_outputs.index == i, 'Node'] = WF_NODES[i - GC] + 1 #save node (from zero-indexed to 1-indexed)
            wind_limit = wind_data.iloc[t, i - GC]
            generator_outputs.loc[generator_outputs.index == i, 'Dispatched Percentage'] = (p_G[i].x / wind_limit) * 100
            generator_outputs.loc[generator_outputs.index == i, 'Type'] = 'Wind'

    for i in range(D):
        print(p_D[i].VarName + ": %.2f MW" % p_D[i].x)
        load_results.loc[load_results.index == i, 'p_D'] = p_D[i].x
        load_results.loc[load_results.index == i, 'Covered Percentage'] = (p_D[i].x / load[load != 0][i]) * 100

    # for k in range(len(constraints)):
    #     print('Dual value {0}: '.format(k+1), dual_values[k])

    print('-----------------------------------------------')

    sum_gen = sum(p_G[n].x for n in range(G))
    sum_load = sum(p_D[n].x for n in range(D))
    lambda_sol = m.Pi[0]
    print("Total load: %.1f MWh" % sum_load)
    print("Total generation: %.1f MWh" % sum_gen)

    print('-----------------------------------------------')
    print("Optimal objective value: %.2f $" % m.objVal)
    print("Lambda: %.2f $/MWh" % m.Pi[0])

    results['gen'] = generator_outputs.copy(deep=True)
    results['demand'] = load_results.copy(deep=True)

else:
    print("Optimization was not successful.")     

P_G[0]: 0.00 MW
P_G[1]: 0.00 MW
P_G[2]: 0.00 MW
P_G[3]: 0.00 MW
P_G[4]: 0.00 MW
P_G[5]: 155.00 MW
P_G[6]: 103.87 MW
P_G[7]: 400.00 MW
P_G[8]: 400.00 MW
P_G[9]: 300.00 MW
P_G[10]: 310.00 MW
P_G[11]: 0.00 MW
P_G[12]: 64.11 MW
P_G[13]: 51.01 MW
P_G[14]: 78.37 MW
P_G[15]: 65.47 MW
P_G[16]: 66.54 MW
P_G[17]: 59.77 MW
P_D[0]: 100.72 MW
P_D[1]: 90.12 MW
P_D[2]: 166.98 MW
P_D[3]: 68.91 MW
P_D[4]: 66.26 MW
P_D[5]: 127.22 MW
P_D[6]: 116.62 MW
P_D[7]: 159.03 MW
P_D[8]: 161.68 MW
P_D[9]: 0.00 MW
P_D[10]: 0.00 MW
P_D[11]: 180.23 MW
P_D[12]: 294.21 MW
P_D[13]: 92.77 MW
P_D[14]: 310.11 MW
P_D[15]: 0.00 MW
P_D[16]: 119.27 MW
-----------------------------------------------
Total load: 2054.1 MWh
Total generation: 2054.1 MWh
-----------------------------------------------
Optimal objective value: 32346.96 $
Lambda: 10.52 $/MWh


In [305]:
# Balancing Model
direction = gb.GRB.MINIMIZE # Min / Max

####
DA_cleared = da_market_price #for hr. 18--> 10.52 $/MWh; day-ahead market clearing price under normal conditions
PS = generator_outputs['p_G'].to_list() # production schedle for all generators within the previous hour
dead_generator_index = 8 # ZERO indexed !
# 
outage_production = PS[dead_generator_index] # Production Schedule of the Failed Generator
#
Cdc = 400 # $/MWh; curtailment cost
low_wf = 0.1 # wind farm power reduction factor ; 10% means the WF produces 90% of the power, scheduled in the DA market
high_wf = 0.15 # wind farm power increase factor ; 15% means the WF produces 115% of the power, scheduled in the DA market

up_regulation_cost_factor = urcf = 0.1 # 10% * generator_cost + generator cost
down_regulation_cost_factor = drcf = 0.13 #  generator cost - 13% * generator_cost
####
m_b = gb.Model() # Create a Gurobi model  

# ============= Variables =============

p_G_u =  m_b.addVars(GC, lb=0, ub=gb.GRB.INFINITY, name="P_G_u") # upward regulation
p_G_d =  m_b.addVars(GC, lb=0, ub=gb.GRB.INFINITY, name="P_G_d") # downward regulation
p_Dc  =  m_b.addVars(D , lb=0, ub=gb.GRB.INFINITY, name="P_Dc") # curtailed demand

# ============= Objective function =============
# Set objective function that minimizes the balancing cost for the system

obj = gb.quicksum(p_G_u[i] * (DA_cleared + urcf * gen_costs['C ($/MWh)'][i]) - p_G_d[i] * (DA_cleared - drcf  * gen_costs['C ($/MWh)'][i]) for i in range(GC)) + gb.quicksum(p_Dc[i] * Cdc for i in range(D)) # curtailemnt into one factor
m_b.setObjective(obj, direction)

# ============= Balance equation ============
m_b.addConstr(gb.quicksum(p_G_u[i] - p_G_d[i] for i in range(GC)) + gb.quicksum(p_Dc[i] for i in range(D)) - outage_production + gb.quicksum(PSi * high_wf for PSi in PS[15:18]) - gb.quicksum(PSi * low_wf for PSi in PS[12:15]) == 0)

# ============= Balancing limits ============
# Downward service limits 
m_b.addConstrs(p_G_d[g] <= PS[g] for g in range(GC)) 

# Upward service limits
m_b.addConstrs(p_G_u[i] <= gen_data['P max MW'][i] - PS[i] for i in range(GC))

# Curtailed demand limits. Ensuring that the crtailed demand is within the total demand of the investigated hour
m_b.addConstrs(p_Dc[i] <= load_results['p_D'][i] for i in range(D)) 

#============= Display and run model =============
m_b.update()
#m_b.display()
m_b.optimize()

print(m_b.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 41 columns and 82 nonzeros
Model fingerprint: 0x085ecac3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 41 rows and 30 columns
Presolve time: 0.01s
Presolved: 1 rows, 11 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.6181067e+04   2.574317e+02   0.000000e+00      0s
       1    4.5323680e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.532367965e+03
4532.367965161845


In [306]:
gen_costs = fnc.read_data('gen_costs')
zero_df = pd.DataFrame(np.zeros(6), columns=['C ($/MWh)'])
gen_costs = pd.concat([gen_costs['C ($/MWh)'], zero_df]).reset_index(drop=True)


In [307]:
# Result anaysis

# add wind farm costs to the df: 
gen_costs = fnc.read_data('gen_costs')
zero_df = pd.DataFrame(np.zeros(6), columns=['C ($/MWh)'])
gen_costs = pd.concat([gen_costs['C ($/MWh)'], zero_df]).reset_index(drop=True)

# Print solutions
if m.status == gb.GRB.OPTIMAL:
    results = {}
    br = pd.DataFrame(data=np.ones(G), columns = ['Node']) ## br = balancing results
    # df setup 
    br['Type'] = 'Conv'
    br['Prod. Cost'] = gen_costs['C ($/MWh)']
    # balancing prices
    #br['Upward Price'] = gen_costs['C ($/MWh)'] *(1 + urcf) 
    #br['Downward Price'] = gen_costs['C ($/MWh)'] *(1 - drcf) 
    br['Node'] = br['Node'].astype(int)
    
    for i in range(G):
        if i < GC:
            br.loc[br.index == i, 'Node'] = gen_data.loc[gen_data.index == i, 'Node'].values #save node
            gen_limit = gen_data.loc[gen_data.index == i, 'P max MW']
            br.loc[br.index == i, 'p_G_u'] = p_G_u[i].x
            br.loc[br.index == i, 'p_G_d'] = p_G_d[i].x
            br.loc[br.index == i, 'p_Dc'] = p_Dc[i].x
        else:
            br.loc[br.index == i, 'Node'] = WF_NODES[i - GC] + 1 # save node (from zero-indexed to 1-indexed)
            wind_limit = wind_data.iloc[t, i - GC]
            br.loc[br.index == i, 'Type'] = 'Wind'
            if i<17:
                br.loc[br.index == i, 'p_Dc'] = p_Dc[i].x
    
    # Actual vs. DA
    br['Scheduled Power'] = PS
    br.loc[br['Type']=='Conv', 'Actual Power'] =  br['Scheduled Power'] + br['p_G_u'] - br['p_G_d']  
    br.loc[br.index[12:15], 'Actual Power'] =  br['Scheduled Power'][12:15] * (1 - low_wf)
    br.loc[br.index[15:18], 'Actual Power'] =  br['Scheduled Power'][15:18] * (1 + high_wf)
    br.loc[br.index[dead_generator_index], 'Actual Power'] = 0

    # Market price for the balancing market
    bm_price = m_b.Pi[0] # Lambda 

    #### PROFITS: ONE PRICE MODEL #####
    #br['Power difference'] = br['Actual Power'] - br['Scheduled Power']
    #br['balancing_revenue'] = bm_price * br['Power difference']
    br['One_price_revenue'] =  da_market_price * br['Scheduled Power'] + bm_price * (br['Actual Power'] - br['Scheduled Power'])# br['balancing_revenue']# 
    br['One_price_profit'] = br['One_price_revenue'] - (br['Actual Power'] * gen_costs['C ($/MWh)'])
  

    #### PROFITS: TWO PRICE MODEL ####
    
  
    ### PROFITS: TWO PRICE MODEL ####
    # Conv. generator total profit from DA and BM
    br['Two_price_profit'] = da_market_price * br['Scheduled Power'] + bm_price * br['p_G_u'] - (br['p_G_u'] + br['Scheduled Power']) * gen_costs['C ($/MWh)']

    # Wind Farm profit from DA and BM:
    br.loc[12:15, 'Two_price_profit'] = da_market_price * br.loc[12:15, 'Scheduled Power'] - bm_price * low_wf * br.loc[12:15, 'Scheduled Power']
    br.loc[15:18, 'Two_price_profit'] = da_market_price * br.loc[15:18, 'Scheduled Power'] + da_market_price * high_wf * br.loc[15:18, 'Scheduled Power']

    # Dead generator losses: 
    br.loc[dead_generator_index, 'Two_price_profit'] = (da_market_price - bm_price ) * PS[dead_generator_index]


    #### PROFITS: TWO PRICE MODEL #####

print('######################################')
print('Lambda: %.2f $/MWh' % bm_price)
print('######################################')
print("Optimal objective value: %.2f $" % m_b.objVal)
print('######################################\n')


print('Balancing Outcomes: ')
print('--------------------')	
br.index = br.index + 1
print(br.round(2).to_string())


######################################
Lambda: 11.61 $/MWh
######################################
Optimal objective value: 4532.37 $
######################################

Balancing Outcomes: 
--------------------
    Node  Type  Prod. Cost   p_G_u  p_G_d  p_Dc  Scheduled Power  Actual Power  One_price_revenue  One_price_profit  Two_price_profit
1      1  Conv       13.32    0.00    0.0   0.0             0.00          0.00               0.00              0.00              0.00
2      2  Conv       13.32    0.00    0.0   0.0             0.00          0.00               0.00              0.00              0.00
3      7  Conv       20.70    0.00    0.0   0.0             0.00          0.00               0.00              0.00              0.00
4     13  Conv       20.93    0.00    0.0   0.0             0.00          0.00               0.00              0.00              0.00
5     15  Conv       26.11    0.00    0.0   0.0             0.00          0.00               0.00              0.00

In [309]:
result_br = pd.concat([br.loc[6:18]])
result_br['Type'] = result_br['Type'].astype(str) + ' ' + result_br.index.astype(str)
result_br.drop(columns=['Node', 'One_price_revenue'], inplace=True)

result_br_round = result_br.round(2)
print(result_br_round.to_string())

result_br_round.columns = ['Gen','$C_g$', r'$p_g^\uparrow$',  r'$p_g^\downarrow$' , r'$p_d^\uparrow$' , r'$p_g^{DA}$' , r'$p_g^{A}$', r'$Profit_g^{I}$' , r'$Profit_g^{II}$']
result_br_round.to_latex('result_br.tex', index=False)

       Type  Prod. Cost   p_G_u  p_G_d  p_Dc  Scheduled Power  Actual Power  One_price_profit  Two_price_profit
6    Conv 6       10.52    0.00    0.0   0.0           155.00        155.00              0.00              0.00
7    Conv 7       10.52   51.13    0.0   0.0           103.87        155.00             55.68             55.68
8    Conv 8        6.02    0.00    0.0   0.0           400.00        400.00           1800.00           1800.00
9    Conv 9        5.47    0.00    0.0   0.0           400.00          0.00           -435.60           -435.60
10  Conv 10        0.00    0.00    0.0   0.0           300.00        300.00           3156.00           3156.00
11  Conv 11       10.52    0.00    0.0   0.0           310.00        310.00              0.00              0.00
12  Conv 12       10.89  339.45    0.0   0.0             0.00        339.45            244.07            244.07
13  Wind 13        0.00     NaN    NaN   0.0            64.11         57.70            599.97           